In [1]:
import os
import pymysql
import pandas as pd

host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    passwd=password,
    db="mimiciiiv14",
    charset='utf8')


In [2]:
'''
admission = pd.read_sql_query("SELECT * FROM `admissions` WHERE 1",conn)
chartevents = pd.read_sql_query("SELECT * FROM `chartevents` WHERE 1",conn)
diagnoses_icd = pd.read_sql_query("SELECT * FROM `diagnoses_icd` WHERE 1",conn)
d_icd_diagnoses = pd.read_sql_query("SELECT * FROM `d_icd_diagnoses` WHERE 1",conn)
d_items = pd.read_sql_query("SELECT * FROM `d_items` WHERE 1",conn)
icustays = pd.read_sql_query("SELECT * FROM `icustays` WHERE 1",conn)
'''

## 總TOPIC:
- 目標:
    預測COPD在此次住院內死亡之COPD可能性
- 方法:
    1. 住院的case有進icu紀錄當作起始點
    2. ***排除掉 : ***
        1. icu天數大於總住院天數
        2. 年齡小於18
        3. 入院診斷描述-文字內沒有「COPD或其他跟COPD有關的描述」
            - elix hauser方式去利用共病症切分data使得他可以針對糖尿病是否有copd
            - 適用embalance落差嚴重
    4. 分群：出院診斷有 COPD/none COPD
    5. 預測在此次ICU內死亡之COPD可能性
- table位置:
    1. icustays內之Intime
    2. ***排除掉 : ***
        1. icu天數(***icustays***/OUTTime-***icustays***/Intime)>總住院天數(***admission***/DISCHTIME-***admission***/ADMITTIME)
        2. 透過入院時間與出生日期可以算出年齡(***admission***/admittime)-(***patients***/dob)
        3. 
    3. 
    4. admission內之diagnosis
    5.
    

#### SCHEMA:
https://mit-lcp.github.io/mimic-schema-spy/


In [ ]:
SELECT d.subject_id, d.hadm_id, gender, dob, dod, seq_num, icd9_code, admittime, dischtime, admission_type, admission_location, discharge_location, insurance, language, 
	religion, marital_status, ethnicity, edregtime, edouttime, diagnosis, a.hospital_expire_flag, has_chartevents_data, icustay_id, dbsource, first_careunit, 
	last_careunit, first_wardid, last_wardid, intime, outtime, los
FROM mimiciii.diagnoses_icd d
	inner join mimiciii.admissions a on d.subject_id = a.subject_id
	inner join mimiciii.icustays icu on d.hadm_id = icu.hadm_id
	inner join mimiciii.patients p on a.subject_id = p.subject_id
-- where icd9_code in ('490', '4910', '4911', '4912', '49120', '49121', '49122', '4918', '4919', '492', '4920', '4928', '494', '4940', '4941', '496', '25000', '25001',  '25002', '25003', '25007', '25004', '25005', '25006')
-- where d.icd9_code BETWEEN '2500' and '2507' --Diabetes
-- where d.icd9_code BETWEEN '4830' and '5837' -- Renal Disease
--  or d.icd9_code in ('5820', '5850','5860', '5880') -- Renal Disease
-- where d.icd9_code in ('4280') --Congestive heart failure
-- where d.icd9_code BETWEEN '1960' and '1991' -- Cancer
where d.icd9_code in ('4019', '4011', '4019', '40210', '40290', '40410', '40490', '4051', '4059') --HTN
where d.icd9_code in ('5851', '5852', '5853', '5854', '5856', '5859' ) --CKD
